# Speedtest data exploration

### How data is  collected

Running speedtest application (?) similar to speedtest.net Calculating upload, download speed + png latency

### How data looks like
Load libraries:

In [1]:
from influxdb import DataFrameClient
from influxdb import InfluxDBClient
import json
import pandas as pd

import datetime 
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.plotly as py
from plotly import tools
init_notebook_mode(connected=True)

Read credentials:

In [2]:
with open('../credentials.json', 'r') as f_credentials:
    credentials_config = json.load(f_credentials)

Set up influxdb connection:

In [3]:
host=credentials_config['influxdb_host']
port=8086
dbname = 'net_speed_md'
client = InfluxDBClient(host, port, '', '', dbname)

Read all the data into pandas dataframe:

In [4]:
client = DataFrameClient(host, port, '', '', dbname)
query_upload = 'SELECT * FROM SPEEDTEST_UPLOAD;'
result_upload= client.query(query_upload)
upload_df = result_upload['SPEEDTEST_UPLOAD']
upload_df.head()

,IP,PI_MAC,PROVIDER,PROVINCE,SK_PI,TEST_SERVER,UPLOAD
2018-10-10 15:45:43.343000064+00:00,204.112.96.66,02-d1-08-c2-95-9b,Bell MTS,MB),1,Shaw Communications (Winnipeg,0.47
2018-10-10 15:51:04.663000064+00:00,204.112.96.66,02-d1-08-c2-95-9b,Bell MTS,MB),1,Shaw Communications (Winnipeg,0.35
2018-10-10 18:40:35.836999936+00:00,204.112.96.66,02-d1-08-c2-95-9b,Bell MTS,MB),1,Rogers (Winnipeg,0.29
2018-10-10 22:38:54.132999936+00:00,204.112.96.66,02-d1-08-c2-95-9b,iperf,iperf,1,clearskystatus.info,698.00
2018-10-10 22:52:08.927000064+00:00,204.112.96.66,02-d1-08-c2-95-9b,iperf,iperf,1,clearskystatus.info,656.00


It's indexed by time, lets reset index and convert time to different format(remove miliseconds)

In [5]:
upload_df.reset_index(level=0, inplace=True)
upload_df['index']=upload_df['index'].dt.strftime('%Y-%m-%d %H:%M:%S')
upload_df['index'] = pd.to_datetime(upload_df['index'])
upload_df['SK_PI']=pd.to_numeric(upload_df['SK_PI'])
upload_df.rename(columns={'index':'time'}, inplace=True)
upload_df.head()

,time,IP,PI_MAC,PROVIDER,PROVINCE,SK_PI,TEST_SERVER,UPLOAD
0,2018-10-10 15:45:43,204.112.96.66,02-d1-08-c2-95-9b,Bell MTS,MB),1,Shaw Communications (Winnipeg,0.47
1,2018-10-10 15:51:04,204.112.96.66,02-d1-08-c2-95-9b,Bell MTS,MB),1,Shaw Communications (Winnipeg,0.35
2,2018-10-10 18:40:35,204.112.96.66,02-d1-08-c2-95-9b,Bell MTS,MB),1,Rogers (Winnipeg,0.29
3,2018-10-10 22:38:54,204.112.96.66,02-d1-08-c2-95-9b,iperf,iperf,1,clearskystatus.info,698.00
4,2018-10-10 22:52:08,204.112.96.66,02-d1-08-c2-95-9b,iperf,iperf,1,clearskystatus.info,656.00


How much data do we have:

In [6]:
upload_df.shape

(3654, 8)

Are there any zeros in Upload speed colum and do they come from NaNs in the original table?

In [7]:
mylist=['UPLOAD']
upload_df[upload_df[mylist].eq(0).all(1)]

,time,IP,PI_MAC,PROVIDER,PROVINCE,SK_PI,TEST_SERVER,UPLOAD
914,2018-11-24 21:36:08,204.112.96.94,02-cd-05-c1-92-3f,Bell MTS,MB),4,Voyageur Internet (Winnipeg,0.0
1061,2018-11-30 14:30:05,204.112.96.94,02-cd-05-c1-92-3f,Bell MTS,MB),4,Shaw Communications (Winnipeg,0.0
1759,2018-12-13 20:42:03,204.112.96.66,02-07-06-80-d4-7c,Bell MTS,MB),7,Commstream Communications (Winnipeg,0.0


In [8]:
import pyodbc
password=credentials_config['mssql_password']
srv=credentials_config['mssql_host']
cnxn = pyodbc.connect(driver='/usr/local/lib/libtdsodbc.so', server=srv,port='1433', database='net_speed_md', uid='cybera_sql', pwd=password)
sql = "SELECT TOP 10 * FROM FCT_SPEEDTEST WHERE UPLOAD='0'OR DOWNLOAD='0' OR PING='0';"
data = pd.read_sql(sql,cnxn)
data

,SK_PI,SK_FILE_PATTERN,DATA_DATE,PROVIDER,IP,TEST_SERVER,PROVINCE,PING,DOWNLOAD,UPLOAD
0,7,19,2018-12-13 20:42:03.723,Bell MTS,204.112.96.66,Commstream Communications (Winnipeg,MB),1800000.000,0.00,0.00
1,4,19,2018-11-24 21:36:08.367,Bell MTS,204.112.96.94,Voyageur Internet (Winnipeg,MB),67.381,1.19,0.00
2,4,19,2018-11-27 05:06:15.177,Bell MTS,204.112.96.94,Shaw Communications (Winnipeg,MB),68.232,0.00,0.65
3,4,19,2018-11-30 14:30:05.207,Bell MTS,204.112.96.94,Shaw Communications (Winnipeg,MB),78.877,1.01,0.00


Something is possibly wrong with these measurments, we wille exclude them for now:

In [9]:
upload_df = upload_df[upload_df.UPLOAD != 0]
upload_df.shape

(3651, 8)

In [10]:
#upload_df.describe(include='all')

### How may data points per device?

In [11]:
points_by_device=upload_df.groupby(['SK_PI']).size().reset_index(name='counts').sort_values('SK_PI')
data = [go.Bar(
            x=points_by_device["SK_PI"],
            y=points_by_device["counts"],
    )]

layout = go.Layout(
        barmode='stack',
        title="Number of data points per device"
    )

fig = go.Figure(data=data, layout=layout)
#print(points_by_device)
iplot(fig)

Device 13 doesn to have any speedtest data?  
Device 5 has the largest amount of datapoints - is it on for a longest time?

#### How many datapoints in the last 4 weeks?

In [23]:
four_weeks_ago = datetime.date.today() - datetime.timedelta(days=28)
upload_df_last4weeks = upload_df[upload_df['time']>four_weeks_ago]
points_by_device_las4weeks = upload_df_last4weeks.groupby(['SK_PI']).size().reset_index(name='counts').sort_values('SK_PI')
merged_results=pd.merge(points_by_device, points_by_device_las4weeks, on='SK_PI', how='outer')
merged_results['counts_y']=merged_results['counts_y'].fillna(0)
merged_results['result']=merged_results['counts_x'].sub(merged_results['counts_y'], axis=0)

In [24]:
trace1 = go.Bar(
            x=merged_results["SK_PI"],
            y=merged_results["counts_y"],
            name='Last 4 weeks',
    )
trace2= go.Bar(
            x=merged_results["SK_PI"],
            y=merged_results['result'],
            name='The rest of the time',
    )
data=[trace1, trace2]
layout = go.Layout(
        barmode='stack',
        title="Number of data points per device: last 4 weeks vs entire time"
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

Devices 1,2,4,6 and 8 have data but not in the last 4 weeks.  
#### When  all the devices started/ended reporting:

In [25]:
start_times=upload_df.groupby('SK_PI')['time'].min()
end_times=upload_df.groupby('SK_PI')['time'].max()
device_numbers=sorted(upload_df['SK_PI'].unique())
#print(device_numbers)
data=[]
for device in device_numbers:
    time_last=end_times[device]
    time_first=start_times[device]
    print("Device: ", device,"  was reporting from ", time_first, " to ", time_last)
    trace = go.Scatter(x=[time_first,time_last],y=[device,device], name = device)
    data.append(trace)
layout = dict(title = "Device reporting times")
fig = go.Figure(data=data, layout=layout)
iplot(fig)

Device:  1   was reporting from  2018-10-10 15:45:43  to  2018-10-11 12:00:09
Device:  2   was reporting from  2018-10-11 15:55:58  to  2018-11-03 16:52:07
Device:  3   was reporting from  2018-10-15 19:36:18  to  2019-01-07 23:48:10
Device:  4   was reporting from  2018-10-27 00:03:48  to  2018-12-05 16:36:12
Device:  5   was reporting from  2018-10-26 18:18:08  to  2019-01-07 23:48:07
Device:  6   was reporting from  2018-10-26 23:53:09  to  2018-10-31 14:52:59
Device:  7   was reporting from  2018-11-11 00:36:13  to  2019-01-07 23:48:04
Device:  8   was reporting from  2018-11-07 18:22:07  to  2018-12-03 16:30:07
Device:  9   was reporting from  2018-11-30 18:12:12  to  2019-01-07 23:48:05
Device:  10   was reporting from  2018-12-12 22:29:40  to  2019-01-07 16:24:08
Device:  11   was reporting from  2018-11-30 18:12:09  to  2019-01-07 23:48:08
Device:  12   was reporting from  2018-11-30 18:12:06  to  2019-01-07 23:48:01
Device:  14   was reporting from  2018-12-05 16:48:41  to  20

### Test servers
Are there any NaNs or empty strings?

In [27]:
print(sum(pd.isnull(upload_df["TEST_SERVER"])))
print(sum(pd.isnull(upload_df["PROVINCE"])))
#print(upload_df[upload_df[["TEST_SERVER","PROVINCE"]].isnull().any(axis=1)])
print(sum(upload_df["TEST_SERVER"] == ''))
print(sum(upload_df["PROVINCE"] == ''))

0
0
0
0


In [17]:
print("There are ",len(upload_df["PROVINCE"].unique()), " provinces: ", upload_df["PROVINCE"].unique(), "\n")
print("There are ",len(upload_df["TEST_SERVER"].unique()), " test servers", upload_df["TEST_SERVER"].unique(), "\n")

pd.options.mode.chained_assignment = None
upload_df["server"] = upload_df["TEST_SERVER"] + " " +upload_df["PROVINCE"]
print("There are ",len(upload_df["server"].unique()), " test servers+provinces", upload_df["server"].unique(), "\n")

There are  5  provinces:  [' MB)' 'iperf' ' AB)' ' SK)' ' ND)'] 

There are  22  test servers ['Shaw Communications (Winnipeg' 'Rogers (Winnipeg' 'clearskystatus.info'
 'LES.NET (Winnipeg' 'Commstream Communications (Winnipeg'
 'Voyageur Internet (Winnipeg' 'TELUS Mobility (Calgary'
 'Axia Connect Ltd (Calgary' 'VOI Network Solutions (Winnipeg'
 'Swift High Speed.com (Winnipeg' 'DataHive (Calgary'
 'Westman Communications (Brandon' 'NetSet Communications (Brandon'
 'Morenet (Morden' 'Valley Fiber (Winkler' 'RFNow Inc. (Virden'
 'BlackSun Inc. (Saskatoon' 'SaskTel (Regina'
 'Access Communications Co-operative Limited (Regina'
 'Bell Mobility (Winnipeg' 'Cybera (Calgary' 'Midco (Minot'] 

There are  22  test servers+provinces ['Shaw Communications (Winnipeg  MB)' 'Rogers (Winnipeg  MB)'
 'clearskystatus.info iperf' 'LES.NET (Winnipeg  MB)'
 'Commstream Communications (Winnipeg  MB)'
 'Voyageur Internet (Winnipeg  MB)' 'TELUS Mobility (Calgary  AB)'
 'Axia Connect Ltd (Calgary  AB)' 'VOI 

#### How many test servers per device?

In [52]:
pd.options.mode.chained_assignment = None
upload_df["server"] = upload_df["TEST_SERVER"] + " " +upload_df["PROVINCE"]
test_servers_per_device = upload_df.groupby('SK_PI').apply(lambda x: x['server'].unique()).apply(pd.Series)
test_servers_per_device['server_count'] = test_servers_per_device.apply(lambda x: x.count(), axis=1)
#print(test_servers_per_device["server_count"].head())
test_server_counts_per_device = pd.Series(test_servers_per_device['server_count']).value_counts().reset_index()
#print(test_server_counts_per_device)
#data = [go.Bar(
#            x=test_servers_per_device.index,
#            y=test_servers_per_device["server_count"],
#    )]
data = [go.Pie(
            labels=test_server_counts_per_device['index'],
            values=test_server_counts_per_device["server_count"],
    )]
layout = go.Layout(
        #barmode='stack',
        title="Number of test servers per device"
    )

fig = go.Figure(data=data, layout=layout)
#print(points_by_device)
iplot(fig)

Devices have different number of test servers: 3-9, how is it defined?

In [50]:
points_by_device_by_server=upload_df.groupby(['SK_PI', 'server']).size().reset_index()
test_servers=upload_df["server"].unique()
data=[]
for server in test_servers:
    serv=[]
    for device in device_numbers:
        by_server=points_by_device_by_server.loc[(points_by_device_by_server['SK_PI']==device)&(points_by_device_by_server['server']==server)]
        if not by_server.empty:
            serv.append(by_server[0].iloc[0])
        else:
            serv.append(0)
    trace = go.Bar(x=device_numbers,y=serv, name = server)
    data.append(trace)
    layout = go.Layout(
        barmode='stack',
        title="Number of data points per device per test server"
    )
fig = go.Figure(data=data, layout=layout)
iplot(fig)

### Providers
Are there any NaNs or empty strings?

In [20]:
print(sum(pd.isnull(upload_df["PROVIDER"])))
print(sum(upload_df["PROVIDER"] == ''))

0
0


In [21]:
print("There are ",len(upload_df["PROVIDER"].unique()), " providers", upload_df["PROVIDER"].unique(), "\n")

There are  11  providers ['Bell MTS' 'iperf' 'Cybera' 'MERLIN' 'High Speed Crow'
 'Cogent Communications' 'Xplornet Communications'
 'Commstream Communications' 'Keewaytinook Okimakanak' 'TeraGo Networks'
 'Manitoba Hydro International'] 



#### How many providers per device?

In [53]:
providers_per_device = upload_df.groupby('SK_PI').apply(lambda x: x["PROVIDER"].unique()).apply(pd.Series)
providers_per_device['provider_count'] = providers_per_device.apply(lambda x: x.count(), axis=1)
#providers_per_device.head()
provider_counts_per_device = pd.Series(providers_per_device['provider_count']).value_counts().reset_index()
#provider_counts_per_device

In [54]:
#data = [go.Bar(
#            x=providers_per_device.index,
#            y=providers_per_device["provider_count"],
#    )]
data = [go.Pie(
            labels=provider_counts_per_device['index'],
            values=provider_counts_per_device["provider_count"],
    )]
layout = go.Layout(
        #barmode='stack',
        title="Number of providers per device"
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

64% of the devices (11 out of 17) use 2 network providers. 1 device uses 3 providers. How is it defined?

In [55]:
points_by_device_by_provider=upload_df.groupby(['SK_PI', 'PROVIDER']).size().reset_index()
providers=upload_df['PROVIDER'].unique()
data=[]
for provider in providers:
    prov=[]
    for device in device_numbers:
        by_provider=points_by_device_by_provider.loc[(points_by_device_by_provider['SK_PI']==device)&(points_by_device_by_provider['PROVIDER']==provider)]
        if not by_provider.empty:
            prov.append(by_provider[0].iloc[0])
        else:
            prov.append(0)
    trace = go.Bar(x=device_numbers,y=prov, name = provider)
    data.append(trace)
    layout = go.Layout(
        barmode='stack',
        title="Number of data points per device per provider"
    )
fig = go.Figure(data=data, layout=layout)
iplot(fig)

### How often data is collected?

Let's ceck device number 5 with the larges number of datapoints and (8 test servers, 2 providers)  and device number 8 with largest number of providers (3) and 4 test servers?

In [29]:
upload_df.loc[upload_df['SK_PI']==5][['time','SK_PI', 'PROVIDER', 'server','UPLOAD']].head(5)

,time,SK_PI,PROVIDER,server,UPLOAD
219,2018-10-26 18:18:08,5,Bell MTS,Voyageur Internet (Winnipeg MB),0.53
220,2018-10-26 20:10:16,5,iperf,clearskystatus.info iperf,514.00
221,2018-10-26 22:00:13,5,Bell MTS,Shaw Communications (Winnipeg MB),0.60
223,2018-10-26 23:52:22,5,iperf,clearskystatus.info iperf,574.00
226,2018-10-27 01:42:06,5,Bell MTS,LES.NET (Winnipeg MB),0.37


In [30]:
upload_df.loc[upload_df['SK_PI']==5][['time','SK_PI', 'PROVIDER', 'server','UPLOAD']].tail(5)

,time,SK_PI,PROVIDER,server,UPLOAD
3597,2019-01-07 09:00:05,5,Bell MTS,Rogers (Winnipeg MB),0.51
3610,2019-01-07 12:42:09,5,Bell MTS,Voyageur Internet (Winnipeg MB),0.53
3630,2019-01-07 16:24:14,5,Bell MTS,Commstream Communications (Winnipeg MB),0.48
3635,2019-01-07 20:06:09,5,Bell MTS,Voyageur Internet (Winnipeg MB),0.48
3646,2019-01-07 23:48:07,5,Bell MTS,Voyageur Internet (Winnipeg MB),0.53


In [31]:
upload_df.loc[upload_df['SK_PI']==8][['time','SK_PI', 'PROVIDER', 'server','UPLOAD']].head(5)

,time,SK_PI,PROVIDER,server,UPLOAD
520,2018-11-07 18:22:07,8,MERLIN,Voyageur Internet (Winnipeg MB),136.88
523,2018-11-07 20:14:27,8,iperf,clearskystatus.info iperf,4034.56
1072,2018-11-30 18:12:12,8,MERLIN,Voyageur Internet (Winnipeg MB),140.37
1077,2018-11-30 21:54:11,8,MERLIN,Commstream Communications (Winnipeg MB),86.70
1087,2018-12-01 01:36:11,8,Commstream Communications,Commstream Communications (Winnipeg MB),85.11


In [32]:
upload_df.loc[upload_df['SK_PI']==5][['time','SK_PI', 'PROVIDER', 'server','UPLOAD']].tail(5)

,time,SK_PI,PROVIDER,server,UPLOAD
3597,2019-01-07 09:00:05,5,Bell MTS,Rogers (Winnipeg MB),0.51
3610,2019-01-07 12:42:09,5,Bell MTS,Voyageur Internet (Winnipeg MB),0.53
3630,2019-01-07 16:24:14,5,Bell MTS,Commstream Communications (Winnipeg MB),0.48
3635,2019-01-07 20:06:09,5,Bell MTS,Voyageur Internet (Winnipeg MB),0.48
3646,2019-01-07 23:48:07,5,Bell MTS,Voyageur Internet (Winnipeg MB),0.53


 Looks like data is collected with 3 hours 42 mins intervals using different test servers. 
 How is it determined which test server to use?
 
 ### To do : 
 Calculate all the intevals?  
 For some devices - plot time series data by test server?

### Median, average, maximum upload speed

In [57]:
def barplot_averages(df,column, title,index_col='SK_PI'):
    max_df = df.groupby(index_col)[column].max().reset_index()
    max_df.columns = [index_col, column+'_MAX']
    med_df = df.groupby(index_col)[column].median().reset_index()
    med_df.columns = [index_col, column+'_MEDIAN']
    avg_df = df.groupby(index_col)[column].mean().reset_index()
    avg_df.columns = [index_col, column+'_MEAN']
    med_max_avg_df=pd.merge(pd.merge(max_df, med_df,on=index_col),avg_df, on=index_col)
    trace1 = go.Bar(
                x=med_max_avg_df[index_col],
                y=med_max_avg_df[column+'_MEAN'],
                name='Mean',
        )
    trace2 = go.Bar(
                x=med_max_avg_df[index_col],
                y=med_max_avg_df[column+'_MAX'],
                name='Max',

        )
    trace3 = go.Bar(
                x=med_max_avg_df[index_col],
                y=med_max_avg_df[column+'_MEDIAN'],
                name='Median',

        )
    data = [trace1, trace2, trace3]
    layout = go.Layout(
           # barmode='stack',
            title=title
            #title="Maximum, mean and median upload speed per device"
        )

    fig = go.Figure(data=data, layout=layout)
    iplot(fig)
barplot_averages(upload_df,'UPLOAD',"Maximum, mean and median upload speed per device")

Device #3 (Cybera office) has the larges spikes, but device 6 have larger median - potentially an issue there.

In [58]:
barplot_averages(upload_df,'UPLOAD',"Maximum, mean and median upload speed per test server",index_col='server')

In [59]:
barplot_averages(upload_df,'UPLOAD',"Maximum, mean and median upload speed per provider",index_col='PROVIDER')

What is 'clearskystatus.info iperf'? It has significantly larger speeds.

### Median, average, maximum download speed

In [131]:
query_download = 'SELECT * FROM SPEEDTEST_DOWNLOAD;'
result_download= client.query(query_download)
download_df = result_download['SPEEDTEST_DOWNLOAD']

download_df.reset_index(level=0, inplace=True)
download_df['index']=download_df['index'].dt.strftime('%Y-%m-%d %H:%M:%S')
download_df['index'] = pd.to_datetime(download_df['index'])
download_df['SK_PI']=pd.to_numeric(download_df['SK_PI'])

download_df.rename(columns={'index':'time'}, inplace=True)
download_df = download_df[download_df.DOWNLOAD != 0]

In [132]:
barplot_averages(download_df,'DOWNLOAD',"Maximum, mean and median download speed per device")

Clearly device #6 has an issue with largest median. Devices #3, #8 #9 having maximum spikes.

In [133]:
download_df["server"] = download_df["TEST_SERVER"] + " " +download_df["PROVINCE"]
barplot_averages(download_df,'DOWNLOAD',"Maximum, mean and median download speed per test server", index_col='server')

In [134]:
barplot_averages(download_df,'DOWNLOAD',"Maximum, mean and median download speed per provider", index_col='PROVIDER')

### Median, average, maximum ping latency

In [60]:
query_ping = 'SELECT * FROM SPEEDTEST_PING;'
result_ping= client.query(query_ping)
ping_df = result_ping['SPEEDTEST_PING']

ping_df.reset_index(level=0, inplace=True)
ping_df['index']=ping_df['index'].dt.strftime('%Y-%m-%d %H:%M:%S')
ping_df['index'] = pd.to_datetime(ping_df['index'])
ping_df['SK_PI']=pd.to_numeric(ping_df['SK_PI'])

ping_df.rename(columns={'index':'time'}, inplace=True)
ping_df = ping_df[ping_df.PING != 1800000.000] ## Removed outier that was found in the ms sql table

In [61]:
barplot_averages(ping_df,'PING',"Maximum, mean and median ping latency per device")

Devices #5 and #16 have largest maximum. Device #10 has large median, average as well.

In [138]:
ping_df["server"] = ping_df["TEST_SERVER"] + " " +ping_df["PROVINCE"]
barplot_averages(ping_df,'PING',"Maximum, mean and median ping latency per test server", index_col='server')

In this case test server 'Swift Highspeed.com' has largest median, average ping latency.

In [63]:
barplot_averages(ping_df,'PING',"Maximum, mean and median ping latency per provider", index_col='PROVIDER')

Bell MTS has largest maximum spikes and TerraGo has largest median/average. 